In [1]:
# Standard library
from collections import defaultdict
from typing import List, Tuple
import importlib.resources as pkg_resources
from multiprocessing.pool import ThreadPool

# Third-party scientific stack
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh

# RDKit
from rdkit import Chem

# PyTorch core
import torch
import torch.nn as nn
import torch.nn.functional as F

# PyTorch Geometric
from torch_geometric.data import Data, Batch, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader as PyGDataLoader

# Polyatomic complexes
from polyatomic_complexes.src.complexes.abstract_complex import AbstractComplex
from polyatomic_complexes.src.complexes import PolyatomicGeometrySMILE

/Users/rahulkhorana/Desktop/polyatomic-experiments/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def set_seed(seed):
    """
    Fix all random seeds for reproducibility across Python, NumPy, and PyTorch.
    """
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(59)

In [3]:
def load_esol_data():
    data_path = pkg_resources.files('polyatomic_complexes.dataset.esol') / 'ESOL.csv'
    df = pd.read_csv(data_path)
    return df

In [4]:
data = load_esol_data()
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles
0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.770,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...
1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.300,Cc1occc1C(=O)Nc2ccccc2
2,citral,-2.579,1,152.237,0,0,4,17.07,-2.060,CC(C)=CCCC(C)=CC(=O)
3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.870,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.330,c1ccsc1
...,...,...,...,...,...,...,...,...,...,...
1123,halothane,-2.608,1,197.381,0,0,0,0.00,-1.710,FC(F)(F)C(Cl)Br
1124,Oxamyl,-0.908,1,219.266,1,0,1,71.00,0.106,CNC(=O)ON=C(SC)C(=O)N(C)C
1125,Thiometon,-3.323,1,246.359,0,0,7,18.46,-3.091,CCSCCSP(=S)(OC)OC
1126,2-Methylbutane,-2.245,1,72.151,0,0,1,0.00,-3.180,CCC(C)C


In [5]:
data.columns

Index(['Compound ID', 'ESOL predicted log solubility in mols per litre',
       'Minimum Degree', 'Molecular Weight', 'Number of H-Bond Donors',
       'Number of Rings', 'Number of Rotatable Bonds', 'Polar Surface Area',
       'measured log solubility in mols per litre', 'smiles'],
      dtype='object')

In [6]:
def build_graph_from_smiles(smile: str, descriptors: List[float], topk_lap: int = 5):
    """
    Builds a PyG Data object with:
      - Node features: chain_0 values (shape [n,1]) + broadcasted descriptors
      - Edge structure and features from get_bonds()
      - Graph-level features: mean/std of chain_1..chain_k + topk eigenvalues of molecule laplacians
    """
    try:
        # 1) Abstract complex
        pg = PolyatomicGeometrySMILE(smile=smile, mode="abstract")
        ac = pg.smiles_to_geom_complex()
        assert isinstance(ac, AbstractComplex)

        # 2) Raw chains
        chains = ac.get_raw_k_chains()
        # chain_0: node-level
        chain0 = chains.get('chain_0')
        if chain0 is None:
            print(f"⚠️ Missing chain_0 for {smile}")
            return None
        n = len(chain0)
        x_node = torch.tensor(chain0, dtype=torch.float32).unsqueeze(1)  # [n,1]
        # graph-level chain stats
        g_stats = []
        for k, arr in chains.items():
            if k == 'chain_0': continue
            a = np.array(arr, dtype=np.float32)
            g_stats.extend([a.mean(), a.std()])
        g_stats = np.array(g_stats, dtype=np.float32)

        # 3) Laplacian eigenvalues (flatten all dims)
        laps = ac.get_laplacians().get('molecule_laplacians', [])
        lap_tuples = []
        for group in laps:
            if isinstance(group, list):
                for item in group:
                    if isinstance(item, tuple) and len(item)==2:
                        lap_tuples.append(item)
            elif isinstance(group, tuple) and len(group)==2:
                lap_tuples.append(group)
        lap_feats = []
        for dim, mat in lap_tuples:
            # convert to sparse if needed
            M = coo_matrix(mat) if not isinstance(mat, coo_matrix) else mat
            k = min(topk_lap, M.shape[0]-1)
            if k <= 0:
                vals = np.zeros(topk_lap, dtype=np.float32)
            else:
                try:
                    vals, _ = eigsh(M, k=k, return_eigenvectors=False)
                except Exception:
                    vals = np.zeros(k, dtype=np.float32)
                if len(vals) < topk_lap:
                    vals = np.pad(vals, (0, topk_lap-len(vals)))
            lap_feats.extend(vals.tolist())
        g_feats = torch.tensor(np.concatenate([g_stats, lap_feats]), dtype=torch.float32)

        # 4) 0-simplices -> node_ids ordering) 0-simplices -> node_ids ordering
        sk = ac.get_skeleta().get('molecule_skeleta', [[]])[0]
        zero = next((lst for dim,lst in sk if dim=='0'), [])
        node_ids = [next(iter(fz))[0] for fz in zero]
        if len(node_ids) != n:
            print(f"⚠️ Node count mismatch for {smile}: {len(node_ids)} vs {n}")
            return None

        # 5) Edges and edge features from bonds
        bonds = ac.get_bonds()  # list of (atom1, atom2, [type, order])
        # Map atom symbols to node indices
        atom_map = defaultdict(list)
        for idx, nid in enumerate(node_ids):
            sym = nid.split('_')[0]
            atom_map[sym].append(idx)

        edge_index_list = []
        edge_attr_list = []
        for a1, a2, (btype, order) in bonds:
            for i in atom_map.get(a1, []):
                for j in atom_map.get(a2, []):
                    edge_index_list.append([i, j])
                    t_int = {'SINGLE': 1, 'DOUBLE': 2, 'TRIPLE': 3}.get(btype, 0)
                    edge_attr_list.append([t_int, float(order)])

        # Fallback: if no bond edges, use 1-simplices from CW-complex
        if not edge_index_list:
            one_list = next((lst for dim, lst in sk if dim == '1'), [])
            for fz in one_list:
                ids = [nid for nid, _ in fz]
                if len(ids) == 2 and ids[0] in node_ids and ids[1] in node_ids:
                    i, j = node_ids[ids[0]], node_ids[ids[1]]
                    edge_index_list.extend([[i, j], [j, i]])
                    # dummy edge_attr for fallback edges
                    edge_attr_list.extend([[0, 0.0], [0, 0.0]])

        edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attr_list, dtype=torch.float32)

        # 6) Combine node+graph descriptors
        desc = torch.tensor(descriptors, dtype=torch.float32)
        desc_feats = desc.unsqueeze(0).expand(n,-1)
        x = torch.cat([x_node, desc_feats], dim=1)

        # 7) Build Data
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
        data.graph_feats = g_feats
        return data

    except Exception as e:
        print(f"❌ Failed {smile}: {e}")
        return None

In [ ]:
def decode_feature_from_frozenset(fzset):
    """
    Expects a frozenset containing a single tuple:
    ('P_3', ('proton', byte_blob))
    Returns a 1D numpy array by decoding the byte blob.
    """
    for _, (_, byte_blob) in fzset:
        try:
            arr = np.frombuffer(byte_blob, dtype=np.float32)
        except (ValueError, TypeError):
            arr = np.frombuffer(byte_blob, dtype=np.float64)
        return arr
    return None

def load_dataset():
    df = load_esol_data()
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    graphs = []
    for _, row in df.iterrows():
        descriptors = [
            row['Molecular Weight'],
            row['Number of H-Bond Donors'],
            row['Number of Rings'],
            row['Number of Rotatable Bonds'],
            row['Polar Surface Area'],
            row['Minimum Degree']
        ]
        g = build_graph_from_smiles(row['smiles'], descriptors)
        if g is not None:
            g.y = torch.tensor([row['measured log solubility in mols per litre']], dtype=torch.float)
            graphs.append(g)
    return graphs


def process_row(args):
    idx, row = args
    try:
        descriptors = [
            row['Molecular Weight'],
            row['Number of H-Bond Donors'],
            row['Number of Rings'],
            row['Number of Rotatable Bonds'],
            row['Polar Surface Area'],
            row['Minimum Degree']
        ]
        graph = build_graph_from_smiles(row['smiles'], descriptors)
        if graph is not None:
            graph.y = torch.tensor([row['measured log solubility in mols per litre']], dtype=torch.float)
        print(f"Success for smile {row['smiles']} at index {idx}")
        return idx, graph
    except Exception as e:
        print(f"Failed on row {idx}: {e}")
        return idx, None

In [8]:
def load_dataset_parallel(df: pd.DataFrame, num_workers=4):
    with ThreadPool(num_workers) as pool:
        results = pool.map(process_row, [(i, row) for i, row in df.iterrows()])
    results = sorted([r for r in results if r[1] is not None], key=lambda x: x[0])
    return [g for _, g in results]

In [ ]:
loading = True
if not loading:
    all_data = load_dataset_parallel(data, num_workers=8)
    torch.save(all_data, 'all_data_esol.pt')

In [10]:
class PolyatomicNet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=64, output_dim=1):
        super().__init__()
        self.embed   = nn.Linear(input_dim, hidden_dim)
        self.conv1   = GCNConv(hidden_dim,   hidden_dim)
        self.conv2   = GCNConv(hidden_dim,   hidden_dim)
        self.readout = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, batch):
        if edge_index.numel() == 0 or edge_index.size(1) == 0:
            n = x.size(0)
            edge_index = torch.stack([torch.arange(n), torch.arange(n)], dim=0).to(x.device)
        x = F.relu(self.embed(x))
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        return self.readout(x).view(-1)

In [11]:
def collate_with_graph_feats(batch_list):
    graph_feats = torch.stack([data.graph_feats for data in batch_list], dim=0)
    for data in batch_list:
        del data.graph_feats
    batched = Batch.from_data_list(batch_list)
    batched.graph_feats = graph_feats
    return batched

In [12]:
def train(model, loader, opt, loss_fn):
    model.train()
    total = 0.0
    for batch in loader:
        opt.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = loss_fn(out, batch.y.view(-1))
        loss.backward()
        opt.step()
        total += loss.item() * batch.num_graphs
    return total / len(loader.dataset)

import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
def compute_metrics_with_ci(trues, preds, n_boot=2000, alpha=0.05, seed=42):
    """
    Compute MAE and RMSE with bootstrap 95% confidence intervals.
    """
    trues = np.array(trues)
    preds = np.array(preds)
    mae = mean_absolute_error(trues, preds)
    rmse = np.sqrt(mean_squared_error(trues, preds))

    rng = np.random.RandomState(seed)
    mae_samples = []
    rmse_samples = []
    n = len(trues)
    for _ in range(n_boot):
        idx = rng.randint(0, n, n)
        t = trues[idx]
        p = preds[idx]
        mae_samples.append(mean_absolute_error(t, p))
        rmse_samples.append(np.sqrt(mean_squared_error(t, p)))

    lower = 100 * (alpha/2)
    upper = 100 * (1 - alpha/2)
    mae_ci = (np.percentile(mae_samples, lower), np.percentile(mae_samples, upper))
    rmse_ci = (np.percentile(rmse_samples, lower), np.percentile(rmse_samples, upper))
    return {'mae': mae, 'mae_ci': mae_ci, 'rmse': rmse, 'rmse_ci': rmse_ci}

def evaluate(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for batch in loader:
            out = model(batch.x, batch.edge_index, batch.batch)
            preds.append(out)
            trues.append(batch.y.view(-1))
    preds = torch.cat(preds)
    trues = torch.cat(trues)
    return torch.sqrt(torch.mean((preds - trues)**2)).item()


def evaluate_with_ci(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for batch in loader:
            out = model(batch.x, batch.edge_index, batch.batch)
            y_true = batch.y.view(-1).cpu().tolist()
            y_pred = out.view(-1).cpu().tolist()
            trues.extend(y_true)
            preds.extend(y_pred)
    metrics = compute_metrics_with_ci(trues, preds)
    return metrics


In [ ]:
loading = True
if loading:
    all_data = torch.load('all_data_esol.pt', weights_only=False)
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    ys = np.array([d.y.item() for d in all_data]).reshape(-1, 1)
    ys_scaled = scaler.fit_transform(ys)
    for i, d in enumerate(all_data):
        d.y = torch.tensor([ys_scaled[i][0]], dtype=torch.float32)

In [14]:
data_list = all_data
train_n = int(0.8*len(data_list))
train_ds, test_ds = torch.utils.data.random_split(data_list, [train_n,len(data_list)-train_n])
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=32)



input_dim = all_data[0].x.size(1)
model     = PolyatomicNet(input_dim=input_dim, hidden_dim=64, output_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn   = nn.MSELoss()

history = {}

for epoch in range(1, 21):
    tr_loss = train(model, train_loader, optimizer, loss_fn)
    metrics = evaluate_with_ci(model, test_loader)
    print(f"Epoch {epoch:02d} | Train Loss: {tr_loss:.4f} | "
          f"Test MAE: {metrics['mae']:.4f} (95% CI [{metrics['mae_ci'][0]:.4f}, {metrics['mae_ci'][1]:.4f}]) | "
          f"Test RMSE: {metrics['rmse']:.4f} (95% CI [{metrics['rmse_ci'][0]:.4f}, {metrics['rmse_ci'][1]:.4f}])")
    history[epoch] = metrics
    

/Users/rahulkhorana/Desktop/polyatomic-experiments/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 01 | Train Loss: 8.3582 | Test MAE: 0.7860 (95% CI [0.6998, 0.8849]) | Test RMSE: 1.0459 (95% CI [0.8601, 1.3152])
Epoch 02 | Train Loss: 0.9171 | Test MAE: 0.6309 (95% CI [0.5742, 0.6907]) | Test RMSE: 0.7768 (95% CI [0.7159, 0.8378])
Epoch 03 | Train Loss: 0.7165 | Test MAE: 0.6883 (95% CI [0.6263, 0.7465]) | Test RMSE: 0.8320 (95% CI [0.7651, 0.8957])
Epoch 04 | Train Loss: 0.7711 | Test MAE: 0.6205 (95% CI [0.5620, 0.6832]) | Test RMSE: 0.7781 (95% CI [0.7007, 0.8700])
Epoch 05 | Train Loss: 0.7883 | Test MAE: 0.6413 (95% CI [0.5830, 0.7038]) | Test RMSE: 0.8017 (95% CI [0.7160, 0.9063])
Epoch 06 | Train Loss: 0.7009 | Test MAE: 0.6943 (95% CI [0.6319, 0.7558]) | Test RMSE: 0.8429 (95% CI [0.7726, 0.9112])
Epoch 07 | Train Loss: 0.6760 | Test MAE: 1.0319 (95% CI [0.9336, 1.1418]) | Test RMSE: 1.2826 (95% CI [1.1514, 1.4369])
Epoch 08 | Train Loss: 0.8425 | Test MAE: 0.5778 (95% CI [0.5256, 0.6317]) | Test RMSE: 0.7123 (95% CI [0.6528, 0.7722])
Epoch 09 | Train Loss: 0.5891 | 

In [15]:
final = history[20]
print("*"*20)
print(f"Test MAE: {final['mae']:.4f} (95% CI [{final['mae_ci'][0]:.4f}, {final['mae_ci'][1]:.4f}])")
print(f"Test RMSE: {final['rmse']:.4f} (95% CI [{final['rmse_ci'][0]:.4f}, {final['rmse_ci'][1]:.4f}])")
print("*"*20)

********************
Test MAE: 0.5788 (95% CI [0.5283, 0.6310])
Test RMSE: 0.6940 (95% CI [0.6381, 0.7493])
********************
